In [15]:
from langchain.agents import initialize_agent, Tool
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

In [16]:
!pip install dotenv


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
load_dotenv()

True

In [18]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [21]:
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model="llama-3.1-8b-instant"   
)

In [ ]:
!pip install -U duckduckgo-search


DuckDuckGo Search

In [22]:
search = DuckDuckGoSearchRun()
search_tool = Tool(
    name="DuckDuckGo Search",
    func=search.run,
    description="Use this tool to perform web searches using DuckDuckGo"
)

tools = [search_tool]



# Initialize agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type="zero-shot-react-description",
    verbose=True
)

# Run query
response = agent.run("What are the latest top ranking movies of 2025?")
print(response)



> Entering new AgentExecutor chain...
Thought: To find the latest top ranking movies of 2025, I should search for information on the top movies of 2025.
Action: DuckDuckGo Search
Action Input: "top movies of 2025"
Observation: ... all, to the start of my Movies Year in Review for 2024! I ’ m your host Jeremy Thomas, and today we ’ re moving to the next 10 in my top ... I won ’ t lie; I didn ’ t expect a DreamWorks movie about a robot, a duck, and a goose to rank among my top five movies of 2024. Top 10 Movies of 2025 That No One Wants ... Get ready for a cynical journey through the most overhyped yet underwhelming movies coming in 2025 and ... ... most anticipated films of 2025 ! From superhero epics to sci-fi adventures, we're counting down the movies ... Top 10 Movies of 2025 That No One Wants In this post, we’ll highlight the top movies of 2025 , including their release dates, where to watch them, and where to download them for offline ...
Thought:Thought: Although the search resu

In [ ]:
!pip install ddgs



Wikipedia Searach

In [23]:
wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=800)

# Initialize Wikipedia tool
wiki_search_tool = Tool(
    name="Wikipedia Search",
    func=wiki.run,
    description="Use this tool to search Wikipedia for concise information."
)

tools = [wiki_search_tool]

# Initialize agent
agent = initialize_agent(
    tools=tools,
    llm=llm,  
    agent_type="zero-shot-react-description",
    verbose=True
)

# Run shorter query
response = agent.run("Elon Musk latest news 2025")
print(response)



> Entering new AgentExecutor chain...
Question: Elon Musk latest news 2025
Thought: I need to find the latest news about Elon Musk in 2025.
Action: Wikipedia Search
Action Input: Elon Musk latest news 2025
Observation: Page: Political activities of Elon Musk
Summary: International businessman Elon Musk has been actively involved in politics, particularly in the United States and Europe, throughout the majority of his career. Despite historically donating to and voting for both Democrats and Republicans, his political contributions have since shifted almost entirely to right-wing candidates and politicians (both centre-right and far-right), outright stating in 2022 that he would no longer support Democrats. In the time since, Musk has become more vocal about his views, frequently promoting conspiracy theories and falsehoods about Democrats, election fraud and immigration. As a result, he has been described as right-wing and conservative, though he rejects the conservative label.
Musk 

In [ ]:
!pip install wikipedia

SQLite Database

In [24]:
import sqlite3

# Connect to database (creates file if it doesn't exist)
connection = sqlite3.connect("example_new.db")
cursor = connection.cursor()

# Create table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS inventory (
        id INTEGER PRIMARY KEY,
        product_name TEXT NOT NULL,
        stock_quantity INTEGER NOT NULL
    )
""")

# Insert sample data
cursor.execute("INSERT INTO inventory (product_name, stock_quantity) VALUES (?, ?)", ("Laptop", 10))
cursor.execute("INSERT INTO inventory (product_name, stock_quantity) VALUES (?, ?)", ("Keyboard", 25))
cursor.execute("INSERT INTO inventory (product_name, stock_quantity) VALUES (?, ?)", ("Mouse", 50))

# Commit changes and close connection
connection.commit()
connection.close()

print("Table created and sample data inserted successfully!")


Table created and sample data inserted successfully!


In [27]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit


In [28]:
db = SQLDatabase.from_uri("sqlite:///example_new.db")


# --- Create SQL toolkit ---
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()  

# --- Initialize agent ---
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type="zero-shot-react-description",  
    verbose=True
)

# --- Run query ---
query1 = "How many units of product Laptop are in stock?"
response1 = agent.run(query1)

print("Response:", response1)



> Entering new AgentExecutor chain...
Thought: I need to know the product with the name Laptop and its stock quantity. 
Action: sql_db_schema
Action Input: products
Observation: Error: table_names {'products'} not found in database
Thought:Action: sql_db_list_tables
Action Input: 
Observation: inventory
Thought:Question: How many units of product Laptop are in stock?
Thought: I need to know the product with the name Laptop and its stock quantity.
Action: sql_db_list_tables
Action Input: 
Observation: inventory
Thought:Action: sql_db_schema
Action Input: inventory
Observation: 
CREATE TABLE inventory (
	id INTEGER, 
	product_name TEXT NOT NULL, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from inventory table:
id	product_name	stock_quantity
1	Laptop	10
2	Keyboard	25
3	Mouse	50
*/
Thought:Thought: I need to check if the product with name 'Laptop' exists in the inventory table.
Action: sql_db_query_checker
Action Input: SELECT * FROM inventory WHERE product_name = 